# Data Preprocessing Tools

## Importing the libraries

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 

## Importing the dataset

In [14]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values #selects all rows, and all columns except last one using iloc to extract features
y = dataset.iloc[:, -1].values  #selects all rows, and last column to extract dependant variable

In [15]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [16]:
print(y)

#Question for future: shape of data, transpose etc. Row vs column?

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [22]:
from sklearn.impute import SimpleImputer
#we are going to replace a missing salary with the average of the salary
imputer  = SimpleImputer(missing_values = np.nan, strategy = 'mean') #How does Imputer work? Can you use specify average by country
imputer.fit(X[:,1:3]) #connects our imputer to our matrix of features X
X[:,1:3] = imputer.transform(X[:,1:3]) #replace with transformed version

In [23]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

In [24]:
#we need to turn strings into numbers to make it easier for ml algorithm to understand, 
#remember, no we want it to know that there is no order for numbers. ie. 0-germ, 1-fr, 2- swed. can cause issues
#one hot encoding fixes this.split country column into a column for each category in the column.
#3 countries - 3 columns
#France = {1,0,0}, Germany = {0,1,0}, Spain = {0,0,1}. No numerical order!
#Also replace booleans with 0 and 1, for no and yes respectively

### Encoding the Independent Variable

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#here we specify the transofmer we are doing, ie, encoding, one hot encoding, on the first column. Passthrough says keep all columns.
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [0] )], remainder ='passthrough')
X = np.array(ct.fit_transform(X)) #we can fit and transform in one go, and assign result to X. Not fit_transform does not return a numpyarray


In [26]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [27]:
#we want to convert no and yes to 0 and 1, we do this using label encoder. This can be used for two classes to assign binary values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y= le.fit_transform(y)#no column, so no need to specify. We do not need np array, so fine to leave as is.


In [29]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [32]:
from sklearn.model_selection import train_test_split
#specify variables, features, dependant, and size split
#recommend 80% in train, 20% in test. Note, it randomly splits
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state =1)


In [33]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [34]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [35]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [36]:
print(y_test)

[0 1]


## Feature Scaling

In [38]:
#should we apply before or after?? Big questions.
#Must be applied after! To prevent information leak.
#Gets mean and std deviation of the feature set. This should not be applied to the test set, as this must be get separate.
#this prevents some features from dominating others.
#we sometimes don't need to apply this.
#two techniques: Standardisation(x-mean)/std dev or Normalisation [x- min(x)]/[(max(x)-min(x)]
#Normalisation is recommended for normal distribution, puts 0 -1
#Standardisation works pretty much all the time, recommended route. puts in range -3 +3
#Apply X_train and X_test. Feature scaling values must be derived from the test set only to prevent information leak.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#Do not apply this to our encoded variables of country and y, will change our variables into nonense.Only apply to numerical values.
X_train[:,3:] = sc.fit_transform(X_train[:,3:]) 
#note: Fit gets mean and std_dev from input values, transform applies the transformation using those values.
X_test[:,3:] = sc.transform(X_test[:,3:]) #transform using the fitted values from train.




In [39]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [40]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
